## **Visualizing fire scars through false color**
Author: Hylaea

Link to the repository: https://github.com/hylaea-miller/eds220-hwk4.git

### **About**
Purpose: what is this notebook about? 
This notebook explores the use of false color imagery to visualize fire scars in satellite images. By manipulating the color channels, we can enhance the visibility of burned areas and analyze the extent and impact of wildfires.

Highlights: List of highlights of analysis (3 or 4 highlights). What do you consider to be the most important aspects of this coding exercise?
About the data: Datasets description
References: Formal references to datasets. 

### **Highlights**
- Utilized false color composites to enhance the visibility of fire scars in satellite imagery.

### **Data**

**Landsat Collection of bands**

Contain simplified collection of bands (red, green, blue, near-infrared and shortwave infrared) from the Landsat Collection 2 Level-2 atmosperically corrected surface reflectance data, collected by the Landsat 8 satellite.

**Palisades and Eaton Dissolved Fire Perimeters**

Contains dissolved fire perimeters/boundaries for Eaton and Palisades fires.

### **References:**
Source: Microsoft Planetary Computer. (2025). Microsoft.com. https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2
Date of accesss: 11/19/2025

Source:  NIFC FIRIS. Palisades and Eaton Dissolved Fire Perimeters (2025). Lacity.org. https://geohub.lacity.org/maps/ad51845ea5fb4eb483bc2a7c38b2370c/about
Date of accesss: 11/19/2025
‌

## Import libraries

In [1]:
# Import libraries
import os              
import numpy as np
import pandas as pd
import xarray as xr   
import geopandas as gpd

## Import data

In [2]:
# Load lansat data
landsat = xr.open_dataset("data/landsat8-2025-02-23-palisades-eaton.nc")

# Load Eaton perimeter data
fp = os.path.join('data', 'Eaton_Perimeter_20250121.shp')
eaton = gpd.read_file(fp)

# Load Palisades perimeter data
fp = os.path.join('data', 'Palisades_Perimeter_20250121.shp')
palisades = gpd.read_file(fp)

## Fire perimeter data exploration

In [19]:
print(eaton.head())
print(palisades.head())

   OBJECTID            type    Shape__Are  Shape__Len  \
0         1  Heat Perimeter   2206.265625  270.199719   
1         2  Heat Perimeter  20710.207031  839.204218   
2         3  Heat Perimeter   3639.238281  250.304502   
3         4  Heat Perimeter   1464.550781  148.106792   
4         5  Heat Perimeter   4132.753906  247.960744   

                                            geometry  
0  POLYGON ((-13146936.686 4051222.067, -13146932...  
1  POLYGON ((-13150835.463 4052713.929, -13150831...  
2  POLYGON ((-13153094.697 4053057.596, -13153113...  
3  POLYGON ((-13145097.740 4053118.235, -13145100...  
4  POLYGON ((-13153131.126 4053196.882, -13153131...  
   OBJECTID            type    Shape__Are   Shape__Len  \
0         1  Heat Perimeter   1182.082031   267.101144   
1         2  Heat Perimeter   2222.488281   185.498783   
2         3  Heat Perimeter     21.011719    22.412814   
3         4  Heat Perimeter    214.992188    76.639180   
4         5  Heat Perimeter  44203.45

In [17]:
print(eaton.dtypes)
print(palisades.dtypes)

OBJECTID         int64
type            object
Shape__Are     float64
Shape__Len     float64
geometry      geometry
dtype: object
OBJECTID         int64
type            object
Shape__Are     float64
Shape__Len     float64
geometry      geometry
dtype: object


In [29]:
print(eaton.crs)
print(palisades.crs.to_dict())

EPSG:3857
{'proj': 'merc', 'a': 6378137, 'b': 6378137, 'lat_ts': 0, 'lon_0': 0, 'x_0': 0, 'y_0': 0, 'k': 1, 'units': 'm', 'nadgrids': '@null', 'wktext': None, 'no_defs': None, 'type': 'crs'}


In [26]:
# Access the CRS of the eaton Geo-dataframe
eaton.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [27]:
# Access the CRS of the palisades Geo-dataframe
palisades.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

### Landsat data exploration

In [7]:
landsat.head()

<xarray.Dataset> Size: 596B
Dimensions:      (y: 5, x: 5)
Coordinates:
  * y            (y) float64 40B 3.799e+06 3.799e+06 ... 3.799e+06 3.799e+06
  * x            (x) float64 40B 3.344e+05 3.344e+05 ... 3.345e+05 3.345e+05
    time         datetime64[ns] 8B ...
Data variables:
    red          (y, x) float32 100B ...
    green        (y, x) float32 100B ...
    blue         (y, x) float32 100B ...
    nir08        (y, x) float32 100B ...
    swir22       (y, x) float32 100B ...
    spatial_ref  int64 8B ...

In [4]:
landsat.info()

xarray.Dataset {
dimensions:
	y = 1418 ;
	x = 2742 ;

variables:
	float64 y(y) ;
		y:units = metre ;
		y:resolution = -30.0 ;
		y:crs = EPSG:32611 ;
		y:axis = Y ;
		y:long_name = y coordinate of projection ;
		y:standard_name = projection_y_coordinate ;
	float64 x(x) ;
		x:units = metre ;
		x:resolution = 30.0 ;
		x:crs = EPSG:32611 ;
		x:axis = X ;
		x:long_name = x coordinate of projection ;
		x:standard_name = projection_x_coordinate ;
	datetime64[ns] time() ;
	float32 red(y, x) ;
		red:grid_mapping = spatial_ref ;
	float32 green(y, x) ;
		green:grid_mapping = spatial_ref ;
	float32 blue(y, x) ;
		blue:grid_mapping = spatial_ref ;
	float32 nir08(y, x) ;
		nir08:grid_mapping = spatial_ref ;
	float32 swir22(y, x) ;
		swir22:grid_mapping = spatial_ref ;
	int64 spatial_ref() ;
		spatial_ref:crs_wkt = PROJCS["WGS 84 / UTM zone 11N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["

## Description of Lansat NetCDF data 
Variables:
red, green, blue, nir08, swir33, spatial_ref

Dimensions:
	y = 1418 ;
	x = 2742 ;
    
    
Coordinates:
(y)
float64
3.799e+06 3.799e+06 ... 3.799e+06


array([3799050., 3799020., 3798990., 3798960., 3798930.])
x
(x)
float64
3.344e+05 3.344e+05 ... 3.345e+05


array([334410., 334440., 334470., 334500., 334530.])